In [1]:
!python -m pip install --upgrade pip

!python -m pip install idc-index

!git clone https://github.com/murong-xu/CADS.git

# Install CADS
%cd CADS
!python -m pip install -U pip
!python -m pip install -e .
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 MB 73.6 MB/s  0:00:016m0:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 172.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 169.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 132.8 MB/s  0:00:00
  Attempting uninstall: pandas━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [duckdb]
    Found existing installation: pandas 2.3.3━━━━━━━━━━━━━━━━━ 1/5 [duckdb]
    Uninstalling pandas-2.3.3:90m╺━━━━━━━━━━━━━━━━━━━━━━━ 2/5 [pandas]
      Successfully uninstalled pandas-2.3.3━━━━━━━━━━━━━━━━━━━ 2/5 [pandas]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [idc-index]/5 [idc-index]data]
Cloning into 'CADS'...
remote: Enumerating objects: 1086, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 1086 (delta 50), reused 77 (delta 27), pack-reused 978 (from 1)
Receiving objects: 100% (1086/1086), 10.52 MiB | 2.71 MiB/s, done

/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///home/jupyter/CADS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ..

In [17]:
!pip uninstall -y numpy pandas
!pip install --no-cache-dir numpy==1.26.4 pandas==2.2.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 49.5 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 172.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]


In [1]:
import os
import sys
import shutil
import subprocess
import traceback
from pathlib import Path
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from idc_index import IDCClient
import SimpleITK as sitk
DICOM_ROOT = Path("./nlst_representative_series")

INPUT_NIFTI_DIR = Path("./ct_nifti")
INPUT_NIFTI_DIR.mkdir(parents=True, exist_ok=True)

CADS_OUT_DIR = Path("./cads_out")
CADS_OUT_DIR.mkdir(parents=True, exist_ok=True)

print("DICOM_ROOT:", DICOM_ROOT.resolve())
print("NIFTI_OUT:", INPUT_NIFTI_DIR.resolve())
print("CADS_OUT:", CADS_OUT_DIR.resolve())

DICOM_ROOT: /home/jupyter/nlst_representative_series
NIFTI_OUT: /home/jupyter/ct_nifti
CADS_OUT: /home/jupyter/cads_out


In [2]:
#!/usr/bin/env python3
from idc_index import IDCClient
"""
Download representative NLST CT series using idc-index.
Generated by NLST CT Acquisition Analysis notebook.
"""

# Initialize IDC client (no authentication required for downloads)
client = IDCClient()

# List of representative SeriesInstanceUIDs
series_uids = ['1.3.6.1.4.1.14519.5.2.1.7009.9004.154032804902040094243713559490', '1.2.840.113654.2.55.314047553821739591629085520556640437258', '1.3.6.1.4.1.14519.5.2.1.7009.9004.156640362884731541203097174511', '1.2.840.113654.2.55.221760170262083317410421571275313269487', '1.2.840.113654.2.55.154596183025472601902508014547512229075', '1.2.840.113654.2.55.223892495254697845073791058058774016570', '1.2.840.113654.2.55.170304785971034342181650359955002661014', '1.3.6.1.4.1.14519.5.2.1.7009.9004.167203483045408857639095733156', '1.3.6.1.4.1.14519.5.2.1.7009.9004.257111818052226587000282847993', '1.3.6.1.4.1.14519.5.2.1.7009.9004.278992675498730944309880982577', '1.3.6.1.4.1.14519.5.2.1.7009.9004.233253643922570418503774287719', '1.3.6.1.4.1.14519.5.2.1.7009.9004.733086154765965465819695855319', '1.3.6.1.4.1.14519.5.2.1.7009.9004.446120418134557663835845900702', '1.3.6.1.4.1.14519.5.2.1.7009.9004.441105416529043363380932190176', '1.2.840.113654.2.55.201655506612925010288892835508800465315', '1.3.6.1.4.1.14519.5.2.1.7009.9004.919216223379924911615465797833', '1.3.6.1.4.1.14519.5.2.1.7009.9004.242079712071558699051299202278', '1.3.6.1.4.1.14519.5.2.1.7009.9004.295310455045405391642755562114', '1.3.6.1.4.1.14519.5.2.1.7009.9004.155682136796606998456894356267', '1.2.840.113654.2.55.330091219482623240054522432469930765363', '1.3.6.1.4.1.14519.5.2.1.7009.9004.201095169663936769948542290144', '1.3.6.1.4.1.14519.5.2.1.7009.9004.293842541786302167508360098817', '1.3.6.1.4.1.14519.5.2.1.7009.9004.617834778270565478282993475869', '1.3.6.1.4.1.14519.5.2.1.7009.9004.195344797572614289877616764543', '1.3.6.1.4.1.14519.5.2.1.7009.9004.261840131606104448047411233035', '1.3.6.1.4.1.14519.5.2.1.7009.9004.102136492893650428036169782021', '1.3.6.1.4.1.14519.5.2.1.7009.9004.243233920577292138649442240317', '1.2.840.113654.2.55.310309040735672349360129308458791106009', '1.2.840.113654.2.55.229477624203342264580867775888850620134', '1.2.840.113654.2.55.129931524337413091771465035428019232784', '1.2.840.113654.2.55.287091428657300092856162782935984503476', '1.3.6.1.4.1.14519.5.2.1.7009.9004.226420772567288035741240692963', '1.2.840.113654.2.55.140563111159321584094131170060719328559', '1.2.840.113654.2.55.48995485399121652606738720093881412631', '1.2.840.113654.2.55.164217626480190476878564152695423139428', '1.2.840.113654.2.55.298854893748984534002057562092121758997', '1.2.840.113654.2.55.279492734292134753419794929571589401481', '1.2.840.113654.2.55.95242108788048571766203081639306629257', '1.2.840.113654.2.55.4873705810622904031106677385023945374', '1.3.6.1.4.1.14519.5.2.1.7009.9004.258670956660823211051324130209', '1.3.6.1.4.1.14519.5.2.1.7009.9004.600393023264914244206809778313', '1.3.6.1.4.1.14519.5.2.1.7009.9004.307853467711025882996136727237']

# Download series
print(f"Downloading {len(series_uids)} representative series...")
client.download_from_selection(
    seriesInstanceUID=series_uids,
    downloadDir="./nlst_representative_series",
    dirTemplate="%collection_id/%PatientID/%SeriesInstanceUID"
)
print("Download complete!")

2026-01-29 15:44:30,641 - Disk size needed: 3.9 GB
2026-01-29 15:44:30,642 - Disk size available: 100.28 GB
2026-01-29 15:44:30,711 - Not using s5cmd sync as the destination folder is empty or sync or progress bar is not requested
2026-01-29 15:44:30,713 - Initial size of the directory: 0 bytes
2026-01-29 15:44:30,714 - Approximate size of the files that need to be downloaded: 3.9 GB


2026-01-29 15:44:38,941 - Successfully downloaded files to /home/jupyter/nlst_representative_series


Download complete!


In [3]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch: 2.10.0+cu128
CUDA available: True


In [4]:
def dicom_series_to_nifti(series_dir: Path, out_dir: Path) -> Path | None:
    """
    Converts a DICOM series folder to CT_*.nii.gz
    Returns output path or None if failed.
    """
    try:
        reader = sitk.ImageSeriesReader()
        dicom_files = reader.GetGDCMSeriesFileNames(str(series_dir))

        if len(dicom_files) == 0:
            return None

        reader.SetFileNames(dicom_files)
        img = reader.Execute()

        # stabiler eindeutiger Name
        uid = hashlib.md5(str(series_dir).encode()).hexdigest()[:12]
        out_path = out_dir / f"CT_{uid}.nii.gz"

        if out_path.exists():
            return out_path

        sitk.WriteImage(img, str(out_path))
        return out_path

    except Exception as e:
        print("Failed:", series_dir, e)
        return None

In [6]:
import hashlib
series_dirs = sorted(p for p in DICOM_ROOT.rglob("*") if p.is_dir())

print("Found potential series dirs:", len(series_dirs))

generated = []

for sd in series_dirs:
    out = dicom_series_to_nifti(sd, INPUT_NIFTI_DIR)
    if out is not None:
        generated.append(out)

print("Generated / found NIFTIs:", len(generated))

Found potential series dirs: 84


GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeriesFileNames (0x55f777872c50): No Series can be found, make sure your restrictions are not too strong

GDCMSeries

Generated / found NIFTIs: 42


In [7]:
import subprocess

assert INPUT_NIFTI_DIR.exists()
assert any(INPUT_NIFTI_DIR.glob("*.nii.gz")), "Keine NIFTIs gefunden!"

cmd = [
    "python", "CADS/cads/scripts/predict_images.py",
    "-in", str(INPUT_NIFTI_DIR),
    "-out", str(CADS_OUT_DIR),
    "-task", "all"
]

print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

Running: python CADS/cads/scripts/predict_images.py -in ct_nifti -out cads_out -task all


Downloading: 100%|██████████| 762M/762M [00:06<00:00, 122MB/s] 


Download finished. Extracting...
Model weights extracted to /home/jupyter/CADS/cads/model_weights/Dataset551_Totalseg251


Downloading: 100%|██████████| 763M/763M [00:04<00:00, 160MB/s] 


Download finished. Extracting...
Model weights extracted to /home/jupyter/CADS/cads/model_weights/Dataset552_Totalseg252


...

100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


_nnUNetv2_predict took 8.714714 seconds to run
Postprocessed group 558 using vertebrae


100%|██████████| 8/8 [00:04<00:00,  1.65it/s]


_nnUNetv2_predict took 9.947684 seconds to run
[ ] subcutaneous_tissue
[ ] muscle
[ ] abdominal_cavity
[?] key='thoracic_cavity' is not set up in the info file
[ ] bones
[?] key='glands' is not set up in the info file
[ ] pericardium
[?] key='breast_implant' is not set up in the info file
[?] key='mediastinum' is not set up in the info file
[ ] spinal_cord
[*] Save cads_out/CT_ffab1a565e74/CT_ffab1a565e74_part_559.nii.gz as uint8
Postprocessed group 559
Restore the segmentation to original format...
From orientation: ('R', 'A', 'S')
To orientation: ('L', 'P', 'S')
Restore the segmentation to original format...
From orientation: ('R', 'A', 'S')
To orientation: ('L', 'P', 'S')
Restore the segmentation to original format...
From orientation: ('R', 'A', 'S')
To orientation: ('L', 'P', 'S')
Restore the segmentation to original format...
From orientation: ('R', 'A', 'S')
To orientation: ('L', 'P', 'S')
Restore the segmentation to original format...
From orientation: ('R', 'A', 'S')
To orient

CompletedProcess(args=['python', 'CADS/cads/scripts/predict_images.py', '-in', 'ct_nifti', '-out', 'cads_out', '-task', 'all'], returncode=0)